# Loading

Create a generator function for importing the file

In [86]:
import urllib.request
import lzma
import base64
import re

def get_dependencies():
    localfile, info = urllib.request.urlretrieve("https://github.com/kevSweet/meta-deps/raw/master/pypi-deps.csv.lzma")
    with lzma.open(localfile, mode='r') as file:
        for line in file:
            package, version, deps = line.decode("utf-8").split('\t')
            deps = base64.b64decode(deps).decode("utf-8")
            deps = eval(deps)
            regex = r'\"*([\w\.]+)[><]?\=?.*\"*'
            
            def cleanDep(s):
                try:
                    match = re.search(regex, s)
                    return match.group(0)
                except:
                    return None
            
            deps = [cleanDep(x) for x in deps if cleanDep(x) is not None]
            
            yield package, deps
        
        


Store as dataframe

In [87]:
import pandas as pd
import numpy as np

rows = np.array([[None,None]])
for package, deps in get_dependencies():
    # Packages with no dependencies get blank string
    if deps:
        args= [[package, dep] for dep in deps]
    else:
        args = [[package, None]]
    rows = np.append(rows, args, axis=0)

        
df = pd.DataFrame(rows, columns=["package", "dependency"]).drop_duplicates()
df.head()

,package,dependency
0,None,None
1,0x10c-asm,None
2,2gis,None
3,3to2,None
4,3to2_py3k,None


In [88]:
from timeit import default_timer

## Redshift

In Redshift we create a single table that will hold one "edge" per row.

In [93]:
import redshift_connector
conn = redshift_connector.connect(
     host='redshift-python-dep.cantgzv41ter.us-east-1.redshift.amazonaws.com',
     database='deps',
     user='awsuser',
     password='REDACTED'
  )
conn.autocommit = True
cursor = conn.cursor()

Create the table

In [91]:
cursor.execute(
"""
    CREATE TABLE dependencies (
        package VARCHAR(150),
        dependency VARCHAR(150) NULL
    )
"""
)

ProgrammingError: {'S': 'ERROR', 'C': '42P07', 'M': 'Relation "dependencies" already exists', 'F': '../src/pg/src/backend/tcop/utility.c', 'L': '204', 'R': 'CheckTableNonExistence'}

Load the data

In [ ]:


# Time it
start = default_timer()

cursor.write_dataframe(df, "dependencies")

# for package, deps in get_dependencies():
#     # Packages with no dependencies get blank string
#     if deps:
#         args = list(product([package], deps))
#     else:
#         args = [(package, "")]
        
#     print(args)
    
#     for tup in args:
#         cursor.execute("INSERT INTO dependencies VALUES (%s, %s)", tup)
#     cursor.executemany("INSERT INTO dependencies VALUES (%s, %s)", args)

end = default_timer()
print(end - start) # Time in seconds, e.g. 5.38091952400282